# Secp256k1

Constants from https://en.bitcoin.it/wiki/Secp256k1

In [6]:
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
F = FiniteField(p)

In [7]:
a = 0
b = 7
E = EllipticCurve(F, [a, b])

In [8]:
G_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
h = 1 # cofactor
G = B = E.lift_x(G_x)

In [9]:
assert(G.order() == 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)

In [13]:
assert(E.order() / G.order() == h)

AssertionError: 

# Key generation

In [14]:
N = FiniteField(G.order())

In [15]:
x = int(N.random_element(1, N.order() - 1))
P = x * G

# ECDSA

In [37]:
import hashlib
def sign (m, d):
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G
    r = int(R.xy()[0]) % N.order()
    assert(r % N.order() != 0)
    s = (e + d*r)/k % N.order()
    return [r,s]
    

In [48]:
sig = sign(b"Hello world", x)

In [49]:
def verify (sig, m, P):
    r, s = sig
    assert(0 < r < N.order())
    assert(0 < s < N.order())
    w = 1 / s % N.order()
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    u1 = e*w % N.order()
    u2 = r*w % N.order()
    R = u1 * G + u2 * P
    v = int(R.xy()[0]) % N.order()
    return v == r

In [51]:
assert(verify(sig, b"Hello world", P))

# ID based signatures

In [43]:
id = int(hashlib.sha256(b"Emil bayes").hexdigest(), 16) % N.order()

In [44]:
Q = id * P

In [45]:
z = id * x % N.order()

In [56]:
sig2 = sign(b"Hello world", z)

In [57]:
verify(sig2, b"Hello world", Q)

True